In [23]:
import requests

In [67]:
response = requests.get(url="http://localhost:8000/question_card/4")

In [68]:
response.text

'{"id":4,"text":"TSA guidelines now prohibit {} on airplanes.","num_gaps":1}'

In [24]:
import json

In [71]:
x = json.loads(response.text)
x["text"]

'TSA guidelines now prohibit {} on airplanes.'

In [25]:
import random

In [167]:
response = requests.post(url="http://localhost:8000/create_player")
PlayerID = json.loads(response.text)
print(PlayerID)

7


In [6]:
response = requests.post(url="http://localhost:8000/create_game")
GameID, QCard = json.loads(response.text)

In [169]:
requests.post(url=f"http://localhost:8000/join_game/{GameID}/{PlayerID}")

<Response [200]>

In [15]:
response = requests.get(url=f"http://localhost:8000/return_qcard/{GameID}")
QCard, NumGaps = json.loads(response.text)
print(QCard, NumGaps)

That’s right, I killed {}. How, you ask? {}. 2


In [176]:
response = requests.get(url=f"http://localhost:8000/return_num_players/{GameID}")
print("number of players:", response.text)

number of players: 1


In [161]:
response = requests.get(url=f"http://localhost:8000/czar/{GameID}/{PlayerID}")
CzarTurn = json.loads(response.text)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [112]:
response = requests.get(url="http://localhost:8000/num_question_cards")
NumQCards = json.loads(response.text)

In [162]:
response = requests.post(url=f"http://localhost:8000/deal_cards/{PlayerID}")
cards = json.loads(response.text)
print("Your Answer Cards:")
for count, card in enumerate(cards):
    print(count + 1, " : ", card)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [130]:
if CzarTurn == True:
    print("you are czar")
else:    
    choice = []
    for index in range(QCard["num_gaps"]):
        choice.append(int(input("choose a card between 1 - 7: ")))
    requests.post(url=f"http://localhost:8000/save_choice/{PlayerID}/{choice}")


CHOOSE!!!!! 2
CHOOSE!!!!! 6
[2, 6]


# Define functions

In [5]:
def CreatePlayer():
    # creates a player in the database
    response = requests.post(url="http://localhost:8000/create_player")
    ID = json.loads(response.text)
    # returns the id of the player
    return ID

In [6]:
def JoinGame(Game, ID):
    # joins the game with id Game
    requests.post(url=f"http://localhost:8000/join_game/{Game}/{ID}")

In [7]:
def ReturnQCard(Game):
    # gets the current question card from the database
    response = requests.get(url=f"http://localhost:8000/return_qcard/{Game}")
    Question, Gaps = json.loads(response.text)
    # returns the information of the question card
    return Question, Gaps

In [8]:
def DealCards(ID):
    # deals 7 answer cards to the player and saves them in the database
    response = requests.post(url=f"http://localhost:8000/deal_cards/{PlayerID}")
    AnsCards = json.loads(response.text)
    # returns the answer cards the player has in the database
    return AnsCards

In [9]:
def SaveChoice(ID, CardChoice):
    # saves the choices of the player in the database
    requests.post(url=f"http://localhost:8000/save_choice/{ID}/{CardChoice}")

In [10]:
def Czar(Game, ID):
    # checks if the player is the czar
    response = requests.get(url=f"http://localhost:8000/czar/{Game}/{ID}")
    Turn = json.loads(response.text)
    return Turn

In [11]:
def GivePoint(Winner):
    # gives a point to the player who was chosen by the czar and saves it in the database
    response = requests.post(url=f"http://localhost:8000/give_points/{Winner}")
    NumPoints = json.loads(response.text)
    # returns the number of points the player who won the round has to check if they have enough points to win the game
    return NumPoints

In [35]:
def PlayerTurn(Question, ID, Gaps):
    # this function is run when the player is not the Czar
    choice = []
    # displays the question card
    print("The Question Card is:")
    print(Question)
    # gets the answer cards and dislplays them
    cards = DealCards(ID)
    print("Your Answer Cards:")
    for count, card in enumerate(cards):
        print(count + 1, " : ", card)
    # allows the user to choose an appropriate number of answer cards
    for index in range(Gaps):
        choice.append(int(input("choose a card between 1 - 7: ")))
    # saves the players choice
    SaveChoice(PlayerID, choice)

In [26]:
def CardsIn(Game):
    # checks if all the players have submitted their answer cards
    choices = []
    # gets the choices submitted by the players
    response = requests.get(url=f"http://localhost:8000/return_choices/{Game}")
    ChoicesCompleted, ChoiceID = json.loads(response.text)
    # loops through each choice and saves it to a list in a nice format
    for choice in ChoiceID:
        TempList = []
        CardInfo = []
        for CardID in choice[1]:
            # gets the answer card text of the answer card with the id found in the choices
            response = requests.get(url=f"http://localhost:8000/answer_card/{CardID}")
            AnsCard = json.loads(response.text)
            TempList.append(AnsCard["text"])
        # saves the text to a nested list
        CardInfo.append(choice[0])
        CardInfo.append(TempList)
        choices.append(CardInfo)
    # In the real thing this would be repeating until all choices are in
    # returns a true statement if all the players have submitted their answer cards as well as the choices
    return ChoicesCompleted, choices

In [27]:
def CzarsTurn(Game, Question):
    # this function will only be run if the player is the Czar
    # checks if all the players have submitted their answer cards
    ChoicesIn, choices = CardsIn(Game)
    if ChoicesIn:
        # displays the question card
        print("The Question Card is:")
        print(Question)
        # displays the choices submitted by the players
        print("The choices are:")
        for count, index in enumerate(choices):
            print(count + 1, " : ", index[1])
            # allows the czar to choose a winner for the round
        PointWinner = int(input("choose a card: "))
        # gives a point to the chosen player
        points = GivePoint(choices[PointWinner - 1][0])
        print(points)
    else:
        # displays this message if not all the players have submitted their answer cards
        print("Not all players have chosen their cards")
    # CZAR IS YET NOT FINISHED

# Start Game

In [28]:
response = requests.post(url="http://localhost:8000/create_game")
GameID, QCard = json.loads(response.text)

## Player 1

In [29]:
CzarTurn = False

In [30]:
PlayerID = CreatePlayer()

In [31]:
JoinGame(GameID, PlayerID)

In [32]:
QCard, NumGaps = ReturnQCard(GameID)
print(QCard)

{} is a slippery slope that leads to {}.


In [33]:
# CzarTurn = Czar(GameID, PlayerID)

In [36]:
if CzarTurn:
    CzarsTurn(GameID, QCard)
else:
    PlayerTurn(QCard, PlayerID, NumGaps)

The Question Card is:
{} is a slippery slope that leads to {}.
Your Answer Cards:
1  :  Multiple stab wounds
2  :  Science
3  :  Dark and mysterious forces beyond our control
4  :  The South
5  :  Autocannibalism
6  :  The Hustle
7  :  Seeing what happens when you lock people in a room with hungry seagulls
choose a card between 1 - 7: 2
choose a card between 1 - 7: 3


## Player 2

In [128]:
CzarTurn = False

In [129]:
choice = []
PlayerID = CreatePlayer()

In [130]:
JoinGame(GameID, PlayerID)

In [131]:
QCard, NumGaps = ReturnQCard(GameID)
print(QCard)

When I am President, I will create the Department of {}.


In [132]:
# CzarTurn = Czar(GameID, PlayerID)

In [133]:
if CzarTurn:
    CzarsTurn(GameID, QCard)
else:
    PlayerTurn(QCard, PlayerID, NumGaps)

The Question Card is:
When I am President, I will create the Department of {}.
Your Answer Cards:
1  :  Land mines
2  :  Pooping in a laptop and closing it
3  :  Vigorous jazz hands
4  :  My abusive boyfriend who really isn’t so bad once you get to know him
5  :  Cardi B
6  :  Sexual peeing
7  :  Accepting the way things are
choose a card between 1 - 7: 4


## Player 3

In [134]:
CzarTurn = False

In [135]:
choice = []
PlayerID = CreatePlayer()

In [136]:
JoinGame(GameID, PlayerID)

In [137]:
QCard, NumGaps = ReturnQCard(GameID)
print(QCard)

When I am President, I will create the Department of {}.


In [138]:
# CzarTurn = Czar(GameID, PlayerID)

In [139]:
if CzarTurn:
    CzarsTurn(GameID, QCard)
else:
    PlayerTurn(QCard, PlayerID, NumGaps)

The Question Card is:
When I am President, I will create the Department of {}.
Your Answer Cards:
1  :  An erection that lasts longer than four hours
2  :  A salad for men that’s made of metal
3  :  Serfdom
4  :  Goblins
5  :  This month’s mass shooting
6  :  Judging everyone
7  :  Flightless birds
choose a card between 1 - 7: 7


## Czar

In [140]:
CzarTurn = True

In [141]:
PlayerID = CreatePlayer()

In [142]:
JoinGame(GameID, PlayerID)

In [143]:
QCard, NumGaps = ReturnQCard(GameID)
print(QCard)

When I am President, I will create the Department of {}.


In [144]:
# CzarTurn = Czar(GameID, PlayerID)

In [145]:
if CzarTurn:
    CzarsTurn(GameID, QCard)
else:
    PlayerTurn(QCard, PlayerID, NumGaps)

The Question Card is:
When I am President, I will create the Department of {}.


KeyError: 'text'

### TODO LIST:
* start game when evreyone is in
* display how many players in the game ---
* need to store game id in player when there are more than one game
* save the Qcard in the table games ---
* in "/deal_cards" give the cards not the card ids in an array ----
* make delte games and players ---
* when i create a game make it so that it saves a QCard ---
* make a function in backend that saves a random QCard ---
* make a function in backend that retruns the current QCard ---
* make cells for the different players in notebook